In [1]:
import cv2
import matplotlib.pyplot as plt
import os
import random
import albumentations as A
import numpy as np

In [2]:
def read_img(img_path):
    return cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

def save_image(image, output_path = "generated_dataset/images"):
    cv2.imwrite(output_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

def display_image(image, title="Image"):
    """
    Display an image using matplotlib. Rembember to use plt.show() to display the image
    """
    plt.imshow(image)
    plt.title(title)
    plt.show()


AUGMENTATION với thư viện albumentations

Các hàm dùng chung cho các phép biến đổi

In [8]:
def transform_image(image, bboxes, transform_fn):
    """
    Áp dụng phép biến đổi lên ảnh và bounding boxes.

    Args:
        image (np.ndarray): Ảnh đầu vào.
        bboxes (list): Danh sách bounding boxes theo định dạng YOLO.
        transform_fn (A.Compose): Phép biến đổi cần áp dụng.
        transform_fn: Hàm biến đổi được lựa chọn.
    Returns:
        np.ndarray: Ảnh đã được biến đổi.
        list: Bounding boxes đã được cập nhật.
    """
    # Xác định phép biến đổi và áp dụng trên ảnh + bounding box
    bbox_params = A.BboxParams(format='yolo')
    transform = A.Compose([transform_fn], bbox_params=bbox_params)

    # Sau khi augmentation
    transformed = transform(image=image, bboxes=bboxes)

    return transformed['image'], transformed['bboxes']


def handle_transform(in_img_path, in_lb_path, out_img_path, out_lb_path, transform_fn):
    """
    Đọc ảnh và nhãn, áp dụng phép biến đổi được lựa chọn và lưu kết quả.

    Args:
        in_img_path (str): Đường dẫn đến ảnh đầu vào.
        in_lb_path (str): Đường dẫn đến tệp nhãn đầu vào.
        out_img_path (str): Đường dẫn để lưu ảnh đã biến đổi.
        out_lb_path (str): Đường dẫn để lưu nhãn đã biến đổi.
        transform_fn: Hàm biến đổi được lựa chọn.
    """
    # Đọc ảnh từ đường dẫn đầu vào
    img = read_img(in_img_path)
    
    with open(in_lb_path, "r") as inf, open(out_lb_path, "w") as outf:
        bboxes = []
        # Đọc và xử lý từng dòng trong tệp nhãn đầu vào để tạo các bounding boxes tương ứng
        for line in inf.readlines():
            values = line.strip().split()
            x_center, y_center, width, height = map(float, values[1:5])
            class_id = int(values[0])
            bboxes.append([x_center, y_center, width, height, class_id])

        # Áp dụng phép biến đổi được lựa chọn
        agm_img, agm_bbox = transform_image(img, bboxes, transform_fn)
        
        # Ghi nhãn đã được biến đổi vào tệp nhãn đầu ra
        for bbox in agm_bbox:
            outf.write(f"{'0'} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")
        
        # Lưu ảnh đã được biến đổi vào tệp ảnh đầu ra
        save_image(agm_img, out_img_path)

def handle_transform_convert(images_folder, labels_folder, output_folder, transform_fn, transform_name):
    """
    Áp dụng phép biến đổi được lựa chọn cho tất cả ảnh và nhãn trong thư mục.

    Args:
        images_folder (str): Thư mục chứa các ảnh đầu vào.
        labels_folder (str): Thư mục chứa các tệp nhãn đầu vào.
    """
    # Định nghĩa các thư mục đầu ra
    out_images_folder = os.path.join(output_folder, transform_name, "images")
    out_labels_folder = os.path.join(output_folder, transform_name, "labels")
    os.makedirs(out_images_folder, exist_ok=True)
    os.makedirs(out_labels_folder, exist_ok=True)
    
    # Duyệt qua tất cả các tệp ảnh trong thư mục đầu vào
    for fileImgName in os.listdir(images_folder):
        if fileImgName.endswith(".jpg"):
            general_name = fileImgName.split('.')[0]
            out_img_name = os.path.join(out_images_folder, f"{general_name}_{transform_name}.jpg")
            in_img_name = os.path.join(images_folder, fileImgName)

            fileLbName = f"{general_name}.txt"
            out_label_name = os.path.join(out_labels_folder, f"{general_name}_{transform_name}.txt")
            in_label_name = os.path.join(labels_folder, fileLbName)

            print(in_img_name)
            print(out_img_name)
            # Gọi hàm handle_transform để xử lý từng ảnh và nhãn
            handle_transform(in_img_name, in_label_name, out_img_name, out_label_name, transform_fn)

Các kỹ thuật cơ bản

In [13]:
# Hàm scale ảnh theo tỉ lệ ngẫu nhiên cho cả chiều rộng và chiều cao
def create_scale_transform_fn(image):
    scale_x = round(random.uniform(0.5, 2), 1)
    scale_y = round(random.uniform(0.5, 2), 1)
    height = int(image.shape[0] * scale_y)
    width = int(image.shape[1] * scale_x)
    return A.Resize(height=height, width=width, p=1)

# Hàm xoay ảnh với góc [-15; 15]
rotate_fn = A.Rotate(limit=15, p=1)


gaussian_noise_fn = A.GaussNoise(p=1)


multiplicative_noise_fn = A.MultiplicativeNoise(multiplier=[0.5, 1.5], per_channel=True, elementwise=True, p=1)

In [17]:
# Áp dụng các phép biến đổi cho tất cả ảnh và nhãn trong thư mục
# Sửa tên folder để đưa vào thư mục mong muốn
# Sửa tên prefix ví dụ "scaled", "rotated" để lưu images, labels thành tên mong muốn

# SCALE thì cần truyền vào kích cỡ image
# for fileImgName in os.listdir("wb_localization_dataset/images/train"):
#     if fileImgName.endswith(".jpg"):
#         img_path = os.path.join("wb_localization_dataset/images/train", fileImgName)
#         img = read_img(img_path)
#         handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train",
#                                  "generated_dataset/scaled_data", create_scale_transform_fn(img), "scaled")

# ROTATE
# handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train",
#                          "generated_dataset/rotated_data", rotate_fn, "rotated")

# NHIỄU GAUSSIAN
# handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train",
#                          "generated_dataset/gaussian_noise_data", gaussian_noise_fn, "gaussian_noise")

# NHIỄU MULTIPLICATIVE
# handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train",
#                          "generated_dataset/multiplicative_noise_data", multiplicative_noise_fn, "multiplicative_noise")

Kỹ thuật nâng cao

Kết hợp các kỹ thuật: rotate, flip, add gauss noise, blur, bright

In [18]:
def create_multi_transform(image):
    scale_x = round(random.uniform(0.5, 2), 1)
    scale_y = round(random.uniform(0.5, 2), 1)
    return A.Compose([
        A.RandomBrightnessContrast(brightness_limit=(-0.5, 0.5), contrast_limit=(-0.5, 0.5), p=0.5),
        A.HorizontalFlip(p=0.5),
        A.Blur(blur_limit=3, p=0.5),
        A.GaussNoise(p=0.5),
        A.Rotate(limit=15, p=0.5),
        A.Resize(height=int(image.shape[0] * scale_y), width=int(image.shape[1] * scale_x), p=0.5),
    ], bbox_params=A.BboxParams(format='yolo'))

In [ ]:
# for fileImgName in os.listdir("wb_localization_dataset/images/train"):
#     if fileImgName.endswith(".jpg"):
#         img_path = os.path.join("wb_localization_dataset/images/train", fileImgName)
#         img = read_img(img_path)
#         handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train",
#                                  "generated_dataset/combine_3_with_others_data/", create_multi_transform_fn(img), "multi")

Augmentation: add noise, rotate, scale cho các ảnh mất cân bằng với rate 0.3 và 0.35 đã phân tích

In [21]:
#Các path để lưu data sau augmentation

biased_35_path = "List_imgs_with_rate/rate_35.txt"
biased_3_path = "List_imgs_with_rate/rate_3.txt"
fd_img_path = "wb_localization_dataset/images/train/"
fd_lb_path = "wb_localization_dataset/labels/train/"
out_img_path_combine_35 = "generated_dataset/combine_3_with_others_data/biased_rate_0.35_data/images/"
out_lb_path_combiine_35 = "generated_dataset/combine_3_with_others_data/biased_rate_0.35_data/labels/"

out_img_path_scale_35 = "generated_dataset/scaled_data/biased_rate_0.35_data/images/"
out_lb_path_scale_35 = "generated_dataset/scaled_data/biased_rate_0.35_data/labels/"

out_img_path_rotate_35 = "generated_dataset/rotated_data/biased_rate_0.35_data/images/"
out_lb_path_rotate_35 = "generated_dataset/rotated_data/biased_rate_0.35_data/labels/"

out_img_path_g_noise_35 = "generated_dataset/noised_data/biased_rate_0.35_data/gaussian_noised_data/images/"
out_lb_path_g_noise_35 = "generated_dataset/noised_data/biased_rate_0.35_data/gaussian_noised_data/labels/"

out_img_path_m_noise_35 = "generated_dataset/noised_data/biased_rate_0.35_data/multiplicative_noised_data/images/"
out_lb_path_m_noise_35 = "generated_dataset/noised_data/biased_rate_0.35_data/multiplicative_noised_data/labels/"

out_img_path_combine_3 = "generated_dataset/combine_3_with_others_data/biased_rate_0.3_data/images/"
out_lb_path_combiine_3 = "generated_dataset/combine_3_with_others_data/biased_rate_0.3_data/labels/"

out_img_path_scale_3 = "generated_dataset/scaled_data/biased_rate_0.3_data/images/"
out_lb_path_scale_3 = "generated_dataset/scaled_data/biased_rate_0.3_data/labels/"

out_img_path_rotate_3 = "generated_dataset/rotated_data/biased_rate_0.3_data/images/"
out_lb_path_rotate_3 = "generated_dataset/rotated_data/biased_rate_0.3_data/labels/"

out_img_path_g_noise_3 = "generated_dataset/noised_data/biased_rate_0.3_data/gaussian_noised_data/images/"
out_lb_path_g_noise_3 = "generated_dataset/noised_data/biased_rate_0.3_data/gaussian_noised_data/labels/"

out_img_path_m_noise_3 = "generated_dataset/noised_data/biased_rate_0.3_data/multiplicative_noised_data/images/"
out_lb_path_m_noise_3 = "generated_dataset/noised_data/biased_rate_0.3_data/multiplicative_noised_data/labels/"

prfx_combine_35 = "_multi_0.35_"
prfx_combine_3 = "_multi_0.3_"

prfx_scale_35 = "_scale_0.35_"
prfx_scale_3 = "_scale_0.3_"

prfx_rotate_35 = "_rotate_0.35_"
prfx_rotate_3 = "_rotate_0.3_"

prfx_g_noise_35 = "_gaussian_noise_0.35_"
prfx_g_noise_3 = "_gaussian_noise_0.3_"

prfx_m_noise_35 = "_multiplicative_noise_0.35_"
prfx_m_noise_3 = "_multiplicative_noise_0.3_"

MULTI GEN

In [23]:
# Đọc các ảnh bị lệnh với rate 0.35 và gọi thực hiện biến đổi kết hợp
with open(biased_35_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        img = read_img(img_path)

        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_combine_35 + prfx_combine_35 + name + ".jpg"
        o_lb_path = out_lb_path_combiine_35 + prfx_combine_35 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, create_multi_transform_fn(img))

# Đọc các ảnh bị lệnh với rate 0.3 và gọi thực hiện biến đổi kết hợp
with open(biased_3_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        img = read_img(img_path)
        
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_combine_3 + prfx_combine_3 + name + ".jpg"
        o_lb_path = out_lb_path_combiine_3 + prfx_combine_3 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, create_multi_transform_fn(img))

SCALE GEN

In [25]:
# Đọc các ảnh bị lệnh với rate 0.35 và gọi thực hiện biến đổi scale
with open(biased_35_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        img = read_img(img_path)

        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_scale_35 + prfx_scale_35 + name + ".jpg"
        o_lb_path = out_lb_path_scale_35 + prfx_scale_35 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, create_scale_transform_fn(img))

# Đọc các ảnh bị lệnh với rate 0.3 và gọi thực hiện biến đổi scale
with open(biased_3_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        img = read_img(img_path)
        
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_scale_3 + prfx_scale_3 + name + ".jpg"
        o_lb_path = out_lb_path_scale_3 + prfx_scale_3 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, create_scale_transform_fn(img))

ROTATE GEN

In [27]:
# Đọc các ảnh bị lệnh với rate 0.35 và gọi thực hiện biến đổi rotate
with open(biased_35_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_rotate_35 + prfx_rotate_35 + name + ".jpg"
        o_lb_path = out_lb_path_rotate_35 + prfx_rotate_35 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, rotate_fn)

# Đọc các ảnh bị lệnh với rate 0.3 và gọi thực hiện biến đổi rotate
with open(biased_3_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_rotate_3 + prfx_rotate_3 + name + ".jpg"
        o_lb_path = out_lb_path_rotate_3 + prfx_rotate_3 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, rotate_fn)

GAUSSIAN NOISE GEN


In [39]:
# Đọc các ảnh bị lệnh với rate 0.35 và gọi thực hiện biến đổi thêm nhiễu gaussian
with open(biased_35_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_g_noise_35 + prfx_g_noise_35 + name + ".jpg"
        o_lb_path = out_lb_path_g_noise_35 + prfx_g_noise_35 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, gaussian_noise_fn)

# Đọc các ảnh bị lệnh với rate 0.3 và gọi thực hiện biến đổi thêm nhiễu gaussian
with open(biased_3_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_g_noise_3 + prfx_g_noise_3 + name + ".jpg"
        o_lb_path = out_lb_path_g_noise_3 + prfx_g_noise_3 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, gaussian_noise_fn)

MULTIPLICATIVE NOISE GEN

In [43]:
# Đọc các ảnh bị lệnh với rate 0.35 và gọi thực hiện biến đổi thêm nhiễu multiplicative
with open(biased_35_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_m_noise_35 + prfx_m_noise_35 + name + ".jpg"
        o_lb_path = out_lb_path_m_noise_35 + prfx_m_noise_35 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, multiplicative_noise_fn)

# Đọc các ảnh bị lệnh với rate 0.3 và gọi thực hiện biến đổi thêm nhiễu multiplicative
with open(biased_3_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path_m_noise_3 + prfx_m_noise_3 + name + ".jpg"
        o_lb_path = out_lb_path_m_noise_3 + prfx_m_noise_3 + name + ".txt"
        # handle_transform(img_path, lb_path, o_img_path, o_lb_path, multiplicative_noise_fn)

Generate translated image

In [ ]:
# Phép biến đổi dịch chuyển
translate_fn = A.ShiftScaleRotate(
    shift_limit_x=0.3, shift_limit_y=0, scale_limit=0, rotate_limit=0,
    border_mode=cv2.BORDER_CONSTANT, value=(0, 0, 0), p=1.0
)

In [12]:
biased_path = "List_imgs_with_rate/rate_35.txt"
fd_img_path = "wb_localization_dataset/images/train/"
fd_lb_path = "wb_localization_dataset/labels/train/"
out_img_path = "generated_dataset/translated_data/translated_biased_rate_0.35_data/images/"
out_lb_path = "generated_dataset/translated_data/translated_biased_rate_0.35_data/labels/"

with open(biased_path, "r") as f:
    for line in f.readlines():
        name = line.split('.')[0]
        img_path = fd_img_path + name + ".jpg"
        lb_path = fd_lb_path + name + ".txt"
        o_img_path = out_img_path + name + ".jpg"
        o_lb_path = out_lb_path + name + ".txt"
        #handle_transform_convert(img_path, lb_path, o_img_path, o_lb_path, translate_fn)


In [ ]:
def gen(images_folder, labels_folder):
    out_folder = "generated_dataset/translated_data/translated_original_data/"
    f_images = out_folder + "images/"
    f_labels = out_folder + "labels/"

    for fileImgName in os.listdir(images_folder):
        if fileImgName.endswith(".jpg"):  
            general_name = fileImgName.split('.')[0]
            out_img_name = f_images + fileImgName
            in_img_name = images_folder + "/" + fileImgName
            #print(out_img_name)

            fileLbName = general_name + ".txt"
            out_label_name = f_labels + fileLbName
            in_label_name = labels_folder + "/" + fileLbName
            print(out_label_name)

            #handle_transform_convert(in_img_name, in_label_name, out_img_name, out_label_name, translate_fn)

gen("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train")

THÊM CÁC LOẠI NHIỄU ĐÃ HỌC với numpy vì thư viện albumentations không hỗ trợ

Định nghĩa các hàm thêm nhiễu

In [ ]:
def add_pepper_noise(image, amount=0.01):
    """
    Thêm nhiễu hạt tiêu vào ảnh.

    Args:
        image (np.ndarray): Ảnh đầu vào.
        amount (float): Tỷ lệ của hạt tiêu.

    Returns:
        np.ndarray: Ảnh với nhiễu hạt tiêu.
    """
    output = np.copy(image)
    num_pepper = np.ceil(amount * image.size)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    output[coords[0], coords[1], :] = 0
    return output

def add_uniform_noise(image, low=0, high=50):
    """
    Thêm nhiễu đồng nhất vào ảnh.

    Args:
        image (np.ndarray): Ảnh đầu vào.
        low (int): Giá trị thấp nhất của phân phối đồng nhất.
        high (int): Giá trị cao nhất của phân phối đồng nhất.

    Returns:
        np.ndarray: Ảnh với nhiễu đồng nhất.
    """
    uniform = np.random.uniform(low, high, image.shape).astype(np.float32)
    noisy = cv2.add(image.astype(np.float32), uniform)
    return np.clip(noisy, 0, 255).astype(np.uint8)

def add_periodic_noise(image, freq_min=5, freq_max=50):
    """
    Thêm nhiễu tuần hoàn vào ảnh.

    Args:
        image (np.ndarray): Ảnh đầu vào.
        freq_min (int): Tần số tối thiểu của nhiễu tuần hoàn.
        freq_max (int): Tần số tối đa của nhiễu tuần hoàn.

    Returns:
        np.ndarray: Ảnh với nhiễu tuần hoàn.
    """
    freq = np.random.uniform(freq_min, freq_max)
    noise = np.zeros_like(image, dtype=np.float32)
    rows, cols = image.shape[:2]
    for i in range(rows):
        for j in range(cols):
            noise[i, j] = 127 * (1 + np.sin(2 * np.pi * freq * i / rows))
    noisy = cv2.addWeighted(image.astype(np.float32), 0.5, noise.astype(np.float32), 0.5, 0)
    return np.clip(noisy, 0, 255).astype(np.uint8)


def add_gamma_noise(image, shape=2.0, scale=30.0):
    """
    Thêm nhiễu gamma vào ảnh.

    Args:
        image (np.ndarray): Ảnh đầu vào.
        shape (float): Tham số shape cho phân phối gamma.
        scale (float): Tham số scale cho phân phối gamma.

    Returns:
        np.ndarray: Ảnh có nhiễu gamma.
    """
    gamma = np.random.gamma(shape, scale, image.shape).astype(np.float32)
    noisy = cv2.add(image.astype(np.float32), gamma)
    return np.clip(noisy, 0, 255).astype(np.uint8)

Các hàm dùng chung

In [ ]:
# Hàm thêm nhiễu dùng chung với nhiễu được lựa chọn
def add_noise(image, noise_type='pepper', **kwargs):
    """
    Thêm nhiễu vào ảnh.

    Args:
        image (np.ndarray): Ảnh đầu vào.
        noise_type (str): Loại nhiễu ('pepper', 'uniform', 'periodic', 'gamma').
        **kwargs: Các tham số cụ thể cho từng loại nhiễu.

    Returns:
        np.ndarray: Ảnh với nhiễu.
    """
    if noise_type == 'pepper':
        return add_pepper_noise(image, **kwargs)
    elif noise_type == 'uniform':
        return add_uniform_noise(image, **kwargs)
    elif noise_type == 'periodic':
        return add_periodic_noise(image, **kwargs)
    elif noise_type == 'gamma':
        return add_gamma_noise(image, **kwargs)
    else:
        raise ValueError("Invalid noise_type. Supported types: 'pepper', 'uniform', 'periodic', 'gamma'.")
    
def noise_transform(image, bboxes, noise_type='pepper', **kwargs):
    """
    Áp dụng phép biến đổi thêm nhiễu cho ảnh và bounding boxes.

    Args:
        image (np.ndarray): Ảnh đầu vào.
        bboxes (list): Danh sách bounding boxes theo định dạng YOLO.
        noise_type (str): Loại nhiễu ('pepper', 'uniform', 'periodic', 'gamma').
        **kwargs: Các tham số cụ thể cho từng loại nhiễu.

    Returns:
        np.ndarray: Ảnh đã được biến đổi.
        list: Bounding boxes đã được cập nhật.
    """
    transformed = add_noise(image, noise_type, **kwargs)
    return transformed, bboxes

In [ ]:
def handle_noise(in_img_path, in_lb_path, out_img_path, out_lb_path, noise_type='pepper', **kwargs):
    img = read_img(in_img_path)

    with open(in_lb_path, "r") as inf, open(out_lb_path, "w") as outf:
        bboxes = []
        for line in inf.readlines():
            values = line.strip().split()
            x_center, y_center, width, height = map(float, values[1:5])
            class_id = int(values[0])
            bboxes.append([x_center, y_center, width, height, class_id])
        # Áp dụng biến đổi nhiễu
        transformed_img, updated_bboxes = noise_transform(img, bboxes, noise_type, **kwargs)
        # Lưu lại các bounding boxes mới vào tệp nhãn
        for bbox in updated_bboxes:
            outf.write(f"{bbox[4]} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")
        # Lưu ảnh đã biến đổi
        save_image(transformed_img, out_img_path)

def handle_noise_convert(images_folder, labels_folder, out_folder, noise_type='pepper', **kwargs):
    f_images = out_folder + "images/"
    f_labels = out_folder + "labels/"

    for file_img_name in os.listdir(images_folder):
        if file_img_name.endswith(".jpg"):  
            general_name = file_img_name.split('.')[0]
            out_img_name = f_images + f"_{noise_type}_noise_" + file_img_name
            in_img_name = images_folder + "/" + file_img_name

            file_lb_name = general_name + ".txt"
            out_label_name = f_labels + f"_{noise_type}_noise_" + file_lb_name
            in_label_name = labels_folder + "/" + file_lb_name

            # Xử lý nhiễu cho từng ảnh và nhãn
            handle_noise(in_img_name, in_label_name, out_img_name, out_label_name, noise_type, **kwargs)

Sinh nhiễu

In [7]:
out_folder = "generated_dataset/noised_data/original_data/uniform_noised_data/"
# handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train", out_folder, noise_type='uniform')

In [11]:
out_folder = "generated_dataset/noised_data/original_data/pepper_noised_data/"
# handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train", out_folder, noise_type='pepper')

In [15]:
out_folder = "generated_dataset/noised_data/original_data/periodic_noised_data/"
# handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train", out_folder, noise_type='periodic')

In [8]:
out_folder = "generated_dataset/noised_data/original_data/gamma_noised_data/"
# handle_transform_convert("wb_localization_dataset/images/train", "wb_localization_dataset/labels/train", out_folder, noise_type='gamma')